# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [1]:
import pandas as pd

# Leer el dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

# 1. Crear un DataFrame nuevo con filtros específicos
filtered_df = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == 'Yes')]
print("Filtered DataFrame:")
print(filtered_df.head())

# 2. Analizar el promedio de total_claim_amount por tipo de póliza y género
response_yes_df = df[df['Response'] == 'Yes']
avg_claim_amount = response_yes_df.groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean().reset_index()
print("Average Total Claim Amount by Policy Type and Gender:")
print(avg_claim_amount)

# Inspeccionar las columnas del DataFrame para verificar los nombres
print(df.columns)

# 3. Analizar el número total de clientes por estado y filtrar estados con más de 500 clientes
state_customer_count = df.groupby('State')['Customer'].count().reset_index()
filtered_states = state_customer_count[state_customer_count['Customer'] > 500]
print("States with more than 500 customers:")
print(filtered_states)

# 4. Encontrar el valor máximo, mínimo y mediano del valor de vida del cliente por nivel educativo y género
clv_agg = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median']).reset_index()
print("Customer Lifetime Value by Education Level and Gender:")
print(clv_agg)

# 5. Bonus: Análisis del número de pólizas vendidas por estado y mes
# Extraer el mes de Effective To Date si no hay una columna Month
df['Effective To Date'] = pd.to_datetime(df['Effective To Date'])
df['Month'] = df['Effective To Date'].dt.month

# a) Crear una tabla donde los meses son columnas y los estados son filas
policies_by_state_month = df.pivot_table(index='State', columns='Month', values='Policy', aggfunc='count', fill_value=0)
print("Policies by State and Month:")
print(policies_by_state_month)

# b) Filtrar los 3 estados con mayor número de pólizas vendidas y crear un nuevo DataFrame
top_states = policies_by_state_month.sum(axis=1).sort_values(ascending=False).head(3).index
top_states_df = policies_by_state_month.loc[top_states]
print("Top 3 States by Policies Sold:")
print(top_states_df)

# 6. Bonus: Análisis de la tasa de respuesta por canal de marketing
response_rate_df = df[df['Response'] == 'Yes']

# Contar las respuestas "Yes" por canal
responses_by_channel = response_rate_df.groupby('Sales Channel')['Response'].count().reset_index(name='Yes_Count')

# Contar el total de respuestas por canal
total_by_channel = df.groupby('Sales Channel')['Response'].count().reset_index(name='Total_Count')

# Merge de ambos DataFrames
merged_df = responses_by_channel.merge(total_by_channel, on='Sales Channel')

# Calcular la tasa de respuesta
merged_df['Response Rate'] = merged_df['Yes_Count'] / merged_df['Total_Count']
print("Response Rate by Sales Channel:")
print(merged_df)

Filtered DataFrame:
     Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
189         189  OK31456  California             11009.130490      Yes   
236         236  YJ16163      Oregon             11009.130490      Yes   
419         419  GW43195      Oregon             25807.063000      Yes   
442         442  IP94270     Arizona             13736.132500      Yes   
587         587  FJ28407  California              5619.689084      Yes   

     Coverage             Education Effective To Date EmploymentStatus Gender  \
189   Premium              Bachelor           1/24/11         Employed      F   
236   Premium              Bachelor           1/24/11         Employed      F   
419  Extended               College           2/13/11         Employed      F   
442   Premium                Master           2/13/11         Disabled      F   
587   Premium  High School or Below           1/26/11       Unemployed      M   

     ...  Number of Open Complaints Number of Po

C:\Users\djjoa\AppData\Local\Temp\ipykernel_4516\1859225048.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Effective To Date'] = pd.to_datetime(df['Effective To Date'])
